## mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Install

In [ ]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
!pip install jiwer
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.8 MB/s eta 0:00:00


## imports

In [ ]:
import pandas as pd
from transformers import MT5Tokenizer, MT5ForConditionalGeneration
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset,DataLoader
from torch.optim import Adam
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
from torch.nn.functional import pad as pad_tokens

## determining environment


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Define paths

In [ ]:
FILE_PATH='/content/drive/MyDrive/Malyalam_Trainining_Data.xlsx'

## Reading Dataset

In [ ]:
df = pd.read_excel(FILE_PATH)
print(df.columns)
df

Index(['ASR OUTPUT', 'Corrected OUTPUT'], dtype='object')


,ASR OUTPUT,Corrected OUTPUT
0,ആയതിനാൽ ചോദ്യോതത്തരങ്ങളുടെ ടൈപ്പ് ചെയ്ത പകർപ്പ...,ആയതിനാൽ ചോദ്യോത്തരങ്ങളുടെ ടൈപ്പ് ചെയ്ത പകർപ്പു...
1,ഹൃദയശൂന്യരായ ഒരുസംഘം നേട്ടിപ്പിടിച്ച നിറത്തോക്...,ഹൃദയശൂന്യരായ ഒരുസംഘം നീട്ടിപ്പിടിച്ച നിറതോക്കി...
2,ആയിരത്തി തൊള്ളായിരത്തി മുപ്പത്തിയഞ്ച് ആഗസ്റ്റ്...,ആയിരത്തി തൊള്ളായിരത്തി മുപ്പത്തിയഞ്ച് ആഗസ്റ്റ്...
3,വിവിധ വകുപ്പുകളിൽ അലക്ഷ്യമായി കൂട്ടിയിട്ടിരിക്...,വിവിധ വകുപ്പുകളിൽ അലക്ഷ്യമായി കൂട്ടിയിട്ടിരിക്...
4,ആയതിനാൽ മേൽ സൂചിപ്പിച്ച കൈപ്പുസ്തകങ്ങളും പ്രസി...,ആയതിനാൽ മേൽ സൂചിപ്പിച്ച കൈപ്പുസ്തകങ്ങളും പ്രസി...
...,...,...
3994,അതിനു മരുന്നോടി ചൊണ്ടിരിക്കും നാളായിരുന്നു,അതിന് മരുന്ന് കുടിച്ചു കൊണ്ടിരിക്കുന്ന ആളായിരു...
3995,എൻറ്റെ ഉമ്മ ഈ വീടുകേള്ളൊക്കെ ജോലി ചയ്തിട്ടാണ് ...,എൻറെ ഉമ്മ ഈ വീടുകളിലൊക്കെ ജോലി ചെയ്തിട്ടാണ് ഞ...
3996,ഞാനും മൂന്ന് സിസ്റ്റഡ്സും മാത്രള്ളൊരു ഫാമിലിയാ...,ഞാനും മൂന്ന് സിസ്റ്റേഴ്സും മാത്രമുള്ളൊരു ഫാമില...
3997,അഎന്തപറയാ ഇവൻ ഞാൻ എം പി വി എസ് ഫസ്റ്റ്യർ പഠിക...,അപ്പോൾ എന്താ പറയാ ഈവൺ ഞാൻ എം ബി ബി എസ് ഫസ്റ്...


## temporary splitting dataset for try out with 50 rows

In [ ]:
# df = df[0:50]
df

,ASR OUTPUT,Corrected OUTPUT
0,ആയതിനാൽ ചോദ്യോതത്തരങ്ങളുടെ ടൈപ്പ് ചെയ്ത പകർപ്പ...,ആയതിനാൽ ചോദ്യോത്തരങ്ങളുടെ ടൈപ്പ് ചെയ്ത പകർപ്പു...
1,ഹൃദയശൂന്യരായ ഒരുസംഘം നേട്ടിപ്പിടിച്ച നിറത്തോക്...,ഹൃദയശൂന്യരായ ഒരുസംഘം നീട്ടിപ്പിടിച്ച നിറതോക്കി...
2,ആയിരത്തി തൊള്ളായിരത്തി മുപ്പത്തിയഞ്ച് ആഗസ്റ്റ്...,ആയിരത്തി തൊള്ളായിരത്തി മുപ്പത്തിയഞ്ച് ആഗസ്റ്റ്...
3,വിവിധ വകുപ്പുകളിൽ അലക്ഷ്യമായി കൂട്ടിയിട്ടിരിക്...,വിവിധ വകുപ്പുകളിൽ അലക്ഷ്യമായി കൂട്ടിയിട്ടിരിക്...
4,ആയതിനാൽ മേൽ സൂചിപ്പിച്ച കൈപ്പുസ്തകങ്ങളും പ്രസി...,ആയതിനാൽ മേൽ സൂചിപ്പിച്ച കൈപ്പുസ്തകങ്ങളും പ്രസി...
...,...,...
3994,അതിനു മരുന്നോടി ചൊണ്ടിരിക്കും നാളായിരുന്നു,അതിന് മരുന്ന് കുടിച്ചു കൊണ്ടിരിക്കുന്ന ആളായിരു...
3995,എൻറ്റെ ഉമ്മ ഈ വീടുകേള്ളൊക്കെ ജോലി ചയ്തിട്ടാണ് ...,എൻറെ ഉമ്മ ഈ വീടുകളിലൊക്കെ ജോലി ചെയ്തിട്ടാണ് ഞ...
3996,ഞാനും മൂന്ന് സിസ്റ്റഡ്സും മാത്രള്ളൊരു ഫാമിലിയാ...,ഞാനും മൂന്ന് സിസ്റ്റേഴ്സും മാത്രമുള്ളൊരു ഫാമില...
3997,അഎന്തപറയാ ഇവൻ ഞാൻ എം പി വി എസ് ഫസ്റ്റ്യർ പഠിക...,അപ്പോൾ എന്താ പറയാ ഈവൺ ഞാൻ എം ബി ബി എസ് ഫസ്റ്...


## Seperating input output

In [ ]:
X,y = df['ASR OUTPUT'],df['Corrected OUTPUT']
X=X.tolist()
y=y.tolist()
print("X",X)
print("y",y)

X ['ആയതിനാൽ ചോദ്യോതത്തരങ്ങളുടെ ടൈപ്പ് ചെയ്ത പകർപ്പുകൾ നിയമസഭയിൽ ലഭ്യമാക്കുന്നതോടൊപ്പം ഓരോ ദിവസത്തെയും ഉത്തരങ്ങൾ ദിവസക്രമത്തിലും മന്ത്രിമാരുടെ ക്രമത്തിലും ലിസ്റ്റിൽ ഉൾപ്പെടുത്തിയിട്ടുള്ള വിധത്തിലും ക്രോഡീകരിച്ച് ചോദ്യം ഉന്നയിക്കുന്നതിൻറ്റെ തലേ ദിവസം മൂന്ന് മണിക്ക് മുൻപായി ചോദ്യവിഭാഗത്തിൻറ്റെ ഇമെയിൽ വിലാസത്തിലേക്ക് നൽകേണ്ടതാണ് ', 'ഹൃദയശൂന്യരായ ഒരുസംഘം നേട്ടിപ്പിടിച്ച നിറത്തോക്കിനുമുന്നിൽ തങ്ങളുടെയും കുടുംബാംഗങ്ങളുടെയും സഹജീവികളുെയും ജീവൻ രക്ഷിക്കാനാവാതെ നിന്ന മുതിർന്ന മനുഷ്യരുടെയും അവരോടൊപ്പം ഉറക്കച്ചടവോടെ അന്തിച്ചുനിന്ന മൂന്നുവയസ്സുകാരിയുടെയും മനസ്സിൽ അവസാനമായി ഉയർന്ന വിചാരമെന്തായിരിക്കുമെന്നു ഓർത്തു നോക്കുക ', 'ആയിരത്തി തൊള്ളായിരത്തി മുപ്പത്തിയഞ്ച് ആഗസ്റ്റ് പതിനാറിന് പന്തളത്തിനടുത്തുള ഉള്ളന്നൂരിൽ മലഞ്ചെരുവിൽ മത്തായിയുടേയും അന്നമ്മയുടേയും നാലാമത്തെ മകനായാണ് ജയിംസ് എന്ന സിറിൾ മാർ മെസേലിയോസ് ജനിച്ചത് ', 'വിവിധ വകുപ്പുകളിൽ അലക്ഷ്യമായി കൂട്ടിയിട്ടിരിക്കുന്ന ഉപയോഗശൂന്യമായ പേപ്പറുകൾ മറ്റ് പാഴ് വസ്തുക്കൾ നീക്കം ചെയ്ത് സെക്ഷനുകൾ വൃത്തിയായി സൂക്ഷിക്കേണ്ടതാണ് ', 'ആയതിനാൽ മേൽ സൂചിപ്പിച്ച കൈപ്പുസ്ത

### Preprocessing data
* After above tryout, found that some white spacing and all issues, so we need to trim the data first


In [ ]:
# trim each element of X and y
X = [sentence.strip() for sentence in X]
y = [sentence.strip() for sentence in y]
print("X",X)
print("y",y)

X ['ആയതിനാൽ ചോദ്യോതത്തരങ്ങളുടെ ടൈപ്പ് ചെയ്ത പകർപ്പുകൾ നിയമസഭയിൽ ലഭ്യമാക്കുന്നതോടൊപ്പം ഓരോ ദിവസത്തെയും ഉത്തരങ്ങൾ ദിവസക്രമത്തിലും മന്ത്രിമാരുടെ ക്രമത്തിലും ലിസ്റ്റിൽ ഉൾപ്പെടുത്തിയിട്ടുള്ള വിധത്തിലും ക്രോഡീകരിച്ച് ചോദ്യം ഉന്നയിക്കുന്നതിൻറ്റെ തലേ ദിവസം മൂന്ന് മണിക്ക് മുൻപായി ചോദ്യവിഭാഗത്തിൻറ്റെ ഇമെയിൽ വിലാസത്തിലേക്ക് നൽകേണ്ടതാണ്', 'ഹൃദയശൂന്യരായ ഒരുസംഘം നേട്ടിപ്പിടിച്ച നിറത്തോക്കിനുമുന്നിൽ തങ്ങളുടെയും കുടുംബാംഗങ്ങളുടെയും സഹജീവികളുെയും ജീവൻ രക്ഷിക്കാനാവാതെ നിന്ന മുതിർന്ന മനുഷ്യരുടെയും അവരോടൊപ്പം ഉറക്കച്ചടവോടെ അന്തിച്ചുനിന്ന മൂന്നുവയസ്സുകാരിയുടെയും മനസ്സിൽ അവസാനമായി ഉയർന്ന വിചാരമെന്തായിരിക്കുമെന്നു ഓർത്തു നോക്കുക', 'ആയിരത്തി തൊള്ളായിരത്തി മുപ്പത്തിയഞ്ച് ആഗസ്റ്റ് പതിനാറിന് പന്തളത്തിനടുത്തുള ഉള്ളന്നൂരിൽ മലഞ്ചെരുവിൽ മത്തായിയുടേയും അന്നമ്മയുടേയും നാലാമത്തെ മകനായാണ് ജയിംസ് എന്ന സിറിൾ മാർ മെസേലിയോസ് ജനിച്ചത്', 'വിവിധ വകുപ്പുകളിൽ അലക്ഷ്യമായി കൂട്ടിയിട്ടിരിക്കുന്ന ഉപയോഗശൂന്യമായ പേപ്പറുകൾ മറ്റ് പാഴ് വസ്തുക്കൾ നീക്കം ചെയ്ത് സെക്ഷനുകൾ വൃത്തിയായി സൂക്ഷിക്കേണ്ടതാണ്', 'ആയതിനാൽ മേൽ സൂചിപ്പിച്ച കൈപ്പുസ്തകങ്ങ

### Train & Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## pretrained mBART-50 from Hugging Face



In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model_name = "facebook/mbart-large-50"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

### tokenizing some sentences to analyze what is the ouptut and how tokenized input output looks like

In [ ]:
sentence = "मैं कल दोपहर 2 बजे मीटिंग में हूँ।"
print("Sentence : ",sentence)
tokenized_sentence = tokenizer(sentence)
print("Tokenized Sentence : ",tokenized_sentence)

Sentence :  मैं कल दोपहर 2 बजे मीटिंग में हूँ।
Tokenized Sentence :  {'input_ids': [250004, 10399, 32587, 233452, 116, 34554, 8415, 72123, 421, 28035, 125, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


### Tokenize Data

In [ ]:
def calculate_max_token_length(X, y, tokenizer):
    # Tokenize all input and target sentences without padding or truncation
    input_lengths = [len(tokenizer.encode(sentence)) for sentence in X]
    target_lengths = [len(tokenizer.encode(sentence)) for sentence in y]

    # Find the maximum length
    max_input_length = max(input_lengths)
    max_target_length = max(target_lengths)

    # Use the larger of the two as the max token length
    max_token_length = max(max_input_length, max_target_length)

    return max_token_length

# Calculate max token length
max_token_length = calculate_max_token_length(X_train + X_test, y_train + y_test, tokenizer)
print(f"Max token length: {max_token_length}")

Max token length: 81


In [ ]:
def tokenize_data(X, y, tokenizer, max_length=max_token_length):
    X = [f"{sentence}" for sentence in X]
    inputs = tokenizer(X, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    targets = tokenizer(y, padding="max_length", max_length=max_length, truncation=True, return_tensors="pt")
    return inputs['input_ids'], targets['input_ids']

# Tokenize training and testing data
X_train_tokenized, y_train_tokenized = tokenize_data(X_train, y_train, tokenizer)
X_test_tokenized, y_test_tokenized = tokenize_data(X_test, y_test, tokenizer)

In [ ]:
print(X_train[0])
print(y_train[0])
print(X_train_tokenized[0])
print(y_train_tokenized[0])
print(X_test_tokenized[0])
print(y_test_tokenized[0])

ഇപ്പോഴും ആ വണ്ടി പറവൂരിലൂടെ ഫട്ഫട് എന്ന് ഓടുന്നുണ്ട്
ഇപ്പോഴും ആ വണ്ടി പറവൂരിലൂടെ ഫട്ഫട് എന്ന് ഓടുന്നുണ്ട്
tensor([250004, 162250,   2569, 183390,  63480,   2903,   5420,   2845,  56823,
         42751,  34136,  23770,  34136,  11079, 175915,  74642,      2,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1])
tensor([250004, 162250,   2569, 183390,  63480,   2903,   5420,   2845,  56823,
         42751,  34136,  23770,  34136,  11079, 175915,  74642,      2,      1,
             

### Managing Acceleration with cuda

In [ ]:
import os
import torch
# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

print("Model and tokenizer loaded successfully!")

Model and tokenizer loaded successfully!


## Dataset

In [ ]:
class ASRDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return (self.inputs[idx]), (self.targets[idx])

train_dataset = ASRDataset(X_train_tokenized,y_train_tokenized)
test_dataset = ASRDataset(X_test_tokenized,y_test_tokenized)

train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=5, shuffle=True)

print("ith item of dataset : ",train_dataset[3])
print("length of dataset : ",len(train_dataset))


ith item of dataset :  (tensor([250004,  23102,   9875,  18356,      6, 123263,  90462,  21471,  88816,
         65000,  11483, 230458, 135847,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1]), tensor([250004,  23102,   9875,  18356,      6, 123263,  90462,  21471,  88816,
         65000,  11483, 230458, 135847,      2,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1

In [ ]:
num_epochs = 3
learning_rate = 5e-5

# Optimizer
optimizer = Adam(model.parameters(), lr=learning_rate)

# Loss function
# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
# import torch.nn.functional as F
# criterion = F.mse_loss

optimizer
criterion

CrossEntropyLoss()

### code to get accuracy on test data

In [ ]:

import numpy as np
from jiwer import wer, cer

def train(model, dataloader, optimizer, device, tokenizer):
    model.train()
    total_loss = 0
    total_tokens = 0
    matched_tokens = 0
    total_wer = 0
    total_cer = 0
    total_sentences = 0
    incorrect_sentences = 0

    for batch_idx, batch in enumerate(dataloader):
        optimizer.zero_grad()

        input_ids = batch[0].to(device)
        attention_mask = (input_ids != tokenizer.pad_token_id).int().to(device)
        labels = batch[1].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Generate model's output
        with torch.no_grad():
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_token_length)

        # Decode input, target, and generated output
        input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
        target_text = tokenizer.decode(labels[0], skip_special_tokens=True)
        generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

        # Convert target and generated text to lowercase
        target_text = target_text.lower()
        generated_text = generated_text.lower()

        # Print tokenized input and output for debugging
        if batch_idx % 10 == 0:
            print(f"Tokenized Input IDs: {input_ids[0]}")
            print(f"Tokenized Target IDs: {labels[0]}")
            print(f"Generated Token IDs: {generated_ids[0]}")

        # Calculate token-level accuracy
        predicted_tokens = generated_ids[0]
        expected_tokens = labels[0]

        # Find the minimum length of the two tensors
        min_length = min(len(predicted_tokens), len(expected_tokens))

        # Truncate both tensors to the minimum length
        predicted_tokens = predicted_tokens[:min_length]
        expected_tokens = expected_tokens[:min_length]

        curr_total_tokens = len(expected_tokens)
        curr_matched_tokens = (predicted_tokens == expected_tokens).sum().item()

        total_tokens += curr_total_tokens
        matched_tokens += curr_matched_tokens

        # Calculate WER and CER
        curr_wer = wer(target_text, generated_text)
        curr_cer = cer(target_text, generated_text)

        total_wer += curr_wer
        total_cer += curr_cer

        # Calculate SER
        if generated_text != target_text:
            incorrect_sentences += 1
        total_sentences += 1

        # Print input, target, and generated output every few batches (e.g., every 10 batches)
        if batch_idx % 10 == 0:
            print(f"Batch {batch_idx}:")
            print(f"Input: {input_text}")
            print(f"Target: {target_text}")
            print(f"Generated: {generated_text}")
            print(f"WER: {curr_wer:.4f}, CER: {curr_cer:.4f}")
            print("-" * 50)

    # Calculate overall metrics
    accuracy = (matched_tokens / total_tokens) * 100.0
    avg_wer = (total_wer / total_sentences) * 100.0
    avg_cer = (total_cer / total_sentences) * 100.0
    ser = (incorrect_sentences / total_sentences) * 100.0

    print(f"Training Metrics:")
    print(f"Accuracy: {accuracy:.2f}%")
    print(f"Average WER: {avg_wer:.2f}%")
    print(f"Average CER: {avg_cer:.2f}%")
    print(f"SER: {ser:.2f}%")
    print("=" * 50)

    return total_loss / len(dataloader)

# Training for a few epochs
for epoch in range(3):
    train_loss = train(model, train_loader, optimizer, device, tokenizer)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

Tokenized Input IDs: tensor([250004, 189093,  15609,   3612,  34141, 163035,   2569,  45536,  21361,
          3023,      6,  17424,  73873,  18560,  37593,  61415,  15609,   5033,
         13427, 130622,      2,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1,
             1,      1,      1,      1,      1,      1,      1,      1,      1],
       device='cuda:0')
Tokenized Target IDs: tensor([250004, 189093,  15609,   3612,  34141, 163035,   2569,  45536,  21361,
          3023,      6,  17424,  73873,  18560,  37593,  61415,  15609,   5033,
         13427, 130622,      2,      1,      1,     

## Save Model

In [ ]:
import os

# Define the directory to save the model and tokenizer
save_directory = "/content/drive/MyDrive/mBART_ASR_malyalam_v2"

os.makedirs(save_directory, exist_ok=True)

# Save the model
model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

print(f"Model and tokenizer saved to {save_directory}")


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Model and tokenizer saved to /content/drive/MyDrive/mBART_ASR_malyalam


In [ ]:
import os
import torch

# Define the directory to save the model
save_directory = "/content/drive/MyDrive/ASR_T5_Model"

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

model_path = os.path.join(save_directory, "flan_t5_asr_model_hindi.pth")
torch.save(model.state_dict(), model_path)

# Save the tokenizer
tokenizer_path = os.path.join(save_directory, "flanT5_tokenizer_hindi")
tokenizer.save_pretrained(tokenizer_path)

print(f"Model saved  {model_path}")
print(f"Tokenizer saved at {tokenizer_path}")


# Evaluate Code

In [ ]:
# state_dict = torch.load(MODEL_PATH)


tokenizer = MBart50TokenizerFast.from_pretrained("/content/drive/MyDrive/mBART_ASR_malyalam_v2")
model = MBartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/mBART_ASR_malyalam_v2")

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Move the model to the desired device (e.g., CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully and moved to device:", device)

Model loaded successfully and moved to device: cpu


In [ ]:
# tokenizer = T5Tokenizer.from_pretrained(TOKENIZER_VARIANT)

In [ ]:
input_file_path = "/content/drive/MyDrive/mal_eval.txt"

EVAL_FILE_PATH = input_file_path
f = open(EVAL_FILE_PATH, "r")
eval_content = f.read()
X_eval = eval_content.split('\n')
print(len(X_eval))
print(max([len(x) for x in X_eval]))


1001
265


In [ ]:
def split_sentence(sentence, max_words):
  splitted_sentences = []
  for i in range(0, len(sentence.split()), max_words):
    curr_sentence = ' '.join(sentence.split()[i:i+max_words])
    splitted_sentences.append(curr_sentence)
  return splitted_sentences


In [ ]:
X_eval_splitted = []
for sentence in X_eval:
  X_eval_splitted.append(split_sentence(sentence,10))
X_eval_splitted

[['ഞാൻ ജയ്ശരീ പി'],
 ['മടുതല ചിന്മയ വിദ്യാലയത്തിലെ'],
 ['മലയാളം അധ്യാപികയാണ്'],
 ['മാനവികത നഷ്ടമാകുന്ന ആധുനികത'],
 ['എന്നാ വിഷയത്തെ മുന്നിർത്തി ചില ചിന്തകൻ'],
 ['നിങ്ങളുമായി പങ്കുവെക്കുന്ാൻ ഞാൻ ആഗ്രഹിക്കുന്നു'],
 ['സ്നേഹമാണകില സാരമൊഴി സ്നേഹത്തിൽ നിന്ന ദിക്കുന്നു ലോകം'],
 ['സ്നേഹത്താൽ വൃദ്ധി നേടുന്നു രോകം'],
 ['മലയാളത്തിൻറ്റെ മഹാകവി കുമാരന ാശാൻറ്റെ വരികളാണ്'],
 ['പരസ്പര സ്നേഹത്തിൻറ്റെ മൂല്യത്തെയാണ് ഈ വരികളിലൂടെ വ്യക്തമാക്കുന്നര്'],
 ['മാനവിക മൂല്യങ്ങൾക്ക് അപജയം സംഭവിക്കുന്നു'],
 ['ഇന്നത്തെ ലോകത്തു നിന്നു കൊണ്ട് ചിന്തിക്കുമോ'],
 ['പല വിധങ്ങളായ പ്രശ്നങ്ങളിൽ പെട്ട് ഉടലുകയാണ്'],
 ['സ്നേഹരാഹിത്യമാണ്'],
 ['പ്രശ്നസഗുലിതമായ ലോകം സൃഷ്ടിക്കുന്നത്'],
 ['സ്നേഹം എന്ന മൂല്യത്തിന് ഇന്ന് വിലയില്ലാതെയായിക്കൊണ്ടിരിക്കുംു'],
 ['വൈവിധ്യങ്ങളുടെ നിറക്കൂട്ടിൽ വിരജിതമാണ് പ്രപഞ്ചം'],
 ['അതിൽ അതിവിശിഷ്ടമായ സ്ഥാനമാണ് മനുഷ്യന് നല്കിയിരിക്കുന്നത്'],
 ['വിവേകംആണ് അതിൻറ്റെ അടിസ്ഥാന'],
 ['അത് കൊണ്ട് തന്നെ കർമ്മവിശ്ുദ്ധിയുടെ ഔന്ിത്യത്തിൽ ഏറ്റവും പ്രതീൃത്തിക്കപ്പെടുന്ന'],
 ['പ്രവർത്തനങ്ങൾ കാഴ്ച വച്ചു'],
 ['ഈ പ്രപഞ്ചത്തെ

In [ ]:

def infer(model, sentence):
  tokenized_sentence = tokenizer(sentence, return_tensors="pt")
  input_ids=(tokenized_sentence['input_ids']).to(device)
  attention_mask=(tokenized_sentence['attention_mask']).to(device)

  output = model.generate(
      input_ids=input_ids,
      attention_mask=attention_mask,
      max_length=512,
      num_beams=5,
      early_stopping=True,
  )

  # Decode the output tokens to text
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  return decoded_output

In [ ]:
def pre(sentence):
  return sentence.lower().strip()
def post(sentence):
  return sentence.upper()

predicted_outputs = []
index = 0
for sentence_batch in X_eval_splitted:
  index += 1
  curr_batch = []
  for sentence in sentence_batch:
    output_text = infer(model,pre(sentence))
    print("   ",output_text)
    cleaned_text = output_text.replace("<|endoftext|>", "")
    cleaned_text = post(cleaned_text)
    curr_batch.append(cleaned_text)
  predicted_output = ' '.join(curr_batch)
  predicted_outputs.append(predicted_output)
  print(index, " / ", len(X_eval_splitted))
  # print(predicted_output)

predicted_outputs

In [ ]:
# join prediected_outputs with '\n'
english_out = '\n'.join(predicted_outputs)
english_out

In [ ]:
OUT_FILE_PATH = '/content/drive/MyDrive/malayalam_eval.txt'
f = open(OUT_FILE_PATH, "w")
f.write(english_out)
f.close()

In [ ]:
tokenizer = MBart50TokenizerFast.from_pretrained("/content/drive/MyDrive/mBART_ASR_malyalam")
model = MBartForConditionalGeneration.from_pretrained("/content/drive/MyDrive/mBART_ASR_malyalam")

In [ ]:
torch.save(model.state_dict(), "mbart-50_asr_malayalam.pth")